In [6]:
from hipscat.pixel_tree.pixel_alignment import align_trees
from hipscat.catalog import Catalog
from tabulate import tabulate

def show_align_percent(catalog1_path, catalog2_path):
    ## Get catalogs and calculate aligment
    catalog1 = Catalog.read_from_hipscat(catalog1_path)
    catalog2 = Catalog.read_from_hipscat(catalog2_path)

    alignment = align_trees(catalog2.pixel_tree, catalog1.pixel_tree)
    aligned_catalog = Catalog(catalog1.catalog_info, alignment.pixel_tree)

    ## Join to original partition data and counts
    just_joined = alignment.pixel_mapping[["primary_Norder", "primary_Npix"]]
    just_joined = just_joined.drop_duplicates().rename(columns={"primary_Norder":"Norder", "primary_Npix":"Npix"}).set_index(["Norder", "Npix"])

    total_rows = catalog2.catalog_info.total_rows
    original_tiles = catalog2.get_pixels()
    total_tiles = len(original_tiles)

    joined_data = original_tiles.set_index(["Norder", "Npix"]).merge(just_joined, how="inner", left_index=True, right_index=True)


    table = [["Partitions",total_tiles,len(joined_data), len(joined_data)/total_tiles*100],
             ["Rows",total_rows,f"{joined_data['num_rows'].sum()}", joined_data['num_rows'].sum()/total_rows*100]]
    print(tabulate(table, headers=["measure", "full catalog", "aligned", "align percent"],
                  colalign=("left", "right", "right")))

In [9]:
## not a huge improvement - sdss doesn't look a lot at the galactic plane, though.

show_align_percent("/data3/epyc/data3/hipscat/catalogs/dr16q_constant/",
                   "/data3/epyc/data3/hipscat/catalogs/sdss_dr16q")

measure       full catalog    aligned    align percent
----------  --------------  ---------  ---------------
Partitions            1032        662          64.1473
Rows             436701643  307457477          70.4045


In [8]:
## Still ~1/3 reduction!

show_align_percent("/data3/epyc/data3/hipscat/catalogs/dr16q_constant/",
                   "/data3/epyc/data3/hipscat/catalogs/ps1/ps1_otmo")

measure       full catalog     aligned    align percent
----------  --------------  ----------  ---------------
Partitions           27161        9291          34.2071
Rows           10560724292  3751804825          35.526


In [7]:
## about a quarter reduction!

show_align_percent("/data3/epyc/data3/hipscat/catalogs/dr16q_constant/",
                   "/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

measure       full catalog    aligned    align percent
----------  --------------  ---------  ---------------
Partitions            2352        554          23.5544
Rows            1234463018  290908232          23.5656
